# Práctica 2a 
Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en
tiempo real a través de una interfaz basada en trackbars o equivalente.

- Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y.
- Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro.
- Hacer escalados uniformes y no uniformes. Es necesario indicar los factores de
escala.

In [9]:
import cv2 as cv
import numpy as np


img = cv.imread('images/cats.png')
if img is None:
    raise FileNotFoundError("No se encontró 'images/cats.png'")
h, w = img.shape[:2]

# Lienzo 2x
CW, CH = w*2, h*2
ox, oy = (CW - w)//2, (CH - h)//2
canvas0 = np.zeros((CH, CW, 3), np.uint8)
canvas0[oy:oy+h, ox:ox+w] = img # imagen en el centro del lienzo

cv.namedWindow("Transformaciones", cv.WINDOW_AUTOSIZE)
def nothing(_): pass

cv.createTrackbar("Tx", "Transformaciones", CW//2, CW, nothing)
cv.createTrackbar("Ty", "Transformaciones", CH//2, CH, nothing)
cv.createTrackbar("Angulo", "Transformaciones", 0, 360, nothing)
cv.createTrackbar("Cx", "Transformaciones", CW//2, CW, nothing)
cv.createTrackbar("Cy", "Transformaciones", CH//2, CH, nothing)
cv.createTrackbar("EscalaX", "Transformaciones", 100, 300, nothing)  # 1.00 = 100
cv.createTrackbar("EscalaY", "Transformaciones", 100, 300, nothing)
cv.createTrackbar("Uniforme", "Transformaciones", 0, 1, nothing)

print("El punto (cx/cy) es el centro de rotación y de escalado. ESC para salir.")

# Transformación afín:
#     T(x) = A·(x - c) + c + t
# donde:
#     A -> rotación y escala combinadas
#     c -> centro de rotación/escalado
#     t -> traslación global
#
# OpenCV aplica:  x' = A·x + b
# donde:
#     b = t + (I - A)·c


t_state = np.array([0.0, 0.0], dtype=np.float32)   # t_state: vector de traslación global (t_x, t_y) que desplaza toda la imagen
prev_c  = np.array([                               # centro anterior de rotación/escalado (c_x, c_y), usado para compensar movimiento
    cv.getTrackbarPos("Cx", "Transformaciones"),
    cv.getTrackbarPos("Cy", "Transformaciones")
], dtype=np.float32)


while True:
    tx_ui = cv.getTrackbarPos("Tx", "Transformaciones") - CW//2
    ty_ui = cv.getTrackbarPos("Ty", "Transformaciones") - CH//2
    ang   = cv.getTrackbarPos("Angulo", "Transformaciones")
    cx    = cv.getTrackbarPos("Cx", "Transformaciones")
    cy    = cv.getTrackbarPos("Cy", "Transformaciones")
    sx    = cv.getTrackbarPos("EscalaX", "Transformaciones") / 100.0
    sy    = cv.getTrackbarPos("EscalaY", "Transformaciones") / 100.0
    if cv.getTrackbarPos("Uniforme", "Transformaciones") == 1:
        sy = sx

    # Rotación y escalado
    rad = np.deg2rad(ang)
    c, s = np.cos(rad), np.sin(rad)
    R = np.array([[ c, -s],
                  [ s,  c]], dtype=np.float32)
    S = np.array([[sx, 0.0],
                  [0.0, sy]], dtype=np.float32)
    A = (R @ S).astype(np.float32)
    I = np.eye(2, dtype=np.float32)

    # Sincronizar t_state con trackbars SOLO si el usuario cambió Tx/Ty
    shown_tx = int(np.clip(round(t_state[0]) + CW//2, 0, CW))
    shown_ty = int(np.clip(round(t_state[1]) + CH//2, 0, CH))
    if (tx_ui + CW//2) != shown_tx or (ty_ui + CH//2) != shown_ty:
        t_state = np.array([tx_ui, ty_ui], dtype=np.float32)

    # Compensar cambio de pivote manteniendo b
    c_now = np.array([cx, cy], dtype=np.float32) # nuevo centro de rotación/escalado
    if not np.allclose(c_now, prev_c):
        b_keep = t_state + (I - A) @ prev_c # valor supuesto con centro anterior
        t_state = b_keep - (I - A) @ c_now # valor con nuevo centro
        cv.setTrackbarPos("Tx", "Transformaciones",
                          int(np.clip(round(t_state[0]) + CW//2, 0, CW)))
        cv.setTrackbarPos("Ty", "Transformaciones",
                          int(np.clip(round(t_state[1]) + CH//2, 0, CH)))
        prev_c = c_now

    b = (t_state + (I - A) @ c_now).astype(np.float32) # traslación final
    M = np.hstack([A, b.reshape(2,1)])  # 2x3
    out = cv.warpAffine(canvas0, M, (CW, CH))

    pc = (A @ c_now + b).astype(int) # pivote transformado p' = A c + b
    disp = out.copy()
    cv.circle(disp, (int(pc[0]), int(pc[1])), 6, (0,0,255), -1)
    cv.putText(disp, f"C=({cx},{cy})", (int(pc[0])+10, int(pc[1])-10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)

    view = disp[oy:oy+h, ox:ox+w]
    cv.imshow("Transformaciones", view)

    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()


El punto (cx/cy) es el centro de rotación y de escalado. ESC para salir.
